# NOTE PATHS HAVE NOT BEEN CHANGED SINCE MOVING INTO NEW FOLDER

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
folder = "data"
files = [folder + f'/DEM_1km_Export_Batch_{i}.csv' for i in range(0,4)]

df_list = [pd.read_csv(file) for file in files]
df_1km = pd.concat(df_list, ignore_index=True)

In [3]:
long = []
lat = []
for i, row in df_1km.iterrows():
    coords = re.search(r'\[(.*?)\]', row['.geo']).group(1).split(',')
    long.append(float(coords[0]))
    lat.append(float(coords[1]))

df_1km['long'] = long
df_1km['lat'] = lat
df_1km.drop(columns=['.geo', 'system:index'], axis=1, inplace=True)

In [4]:
print("rows by columns:", df_1km.shape)
print("columns:", df_1km.columns)
print("longitude range:", max(df_1km['long']), min(df_1km['long']))
print("latitude range:", max(df_1km['lat']), min(df_1km['lat']))
print("elevation range:", max(df_1km['elevation']), min(df_1km['elevation']))

rows by columns: (3742312, 3)
columns: Index(['elevation', 'long', 'lat'], dtype='object')
longitude range: 153.46369156255847 137.91385399444954
latitude range: -15.096188349628559 -38.45238573673612
elevation range: 2135.8901 -44.925156


In [5]:
lapse_rate = 0.0065 # deg C per m
sea_level_temp = 19 # deg C - along east coast, using Sydney as reference
err_std_dev = 0.5 # deg C - taking into account errors for lapse rate and sea level temp

def elevation_to_temp(elevations): # elevation in meters
    temps = []
    for elevation in elevations:
        err = np.random.normal(0, err_std_dev)
        temp = sea_level_temp - (lapse_rate * elevation) + err
        temps.append(temp)
    return temps

In [6]:
df_1km['generated_temp'] = elevation_to_temp(df_1km['elevation'])
df_1km.to_csv(folder + '/1km_data.csv', index=False)